In [ ]:
import json
from io import open
import torch
from torch.utils.data import Dataset
from transformers import BertModel, BertTokenizer
from transformers import AdamW
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [ ]:
%pip install transformers
%pip install sentencepiece
%pip install imbalanced-learn

In [ ]:
file_path = "/kaggle/input/diplomacy/train.jsonl"
train_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        # Parse the line as JSON
        train_data[i] = json.loads(line)
        i+=1

In [2]:
file_path = "/kaggle/input/diplomacy/test.jsonl"
test_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        # Parse the line as JSON
        test_data[i] = json.loads(line)
        i+=1

In [3]:
messages_26= train_data[36]["messages"]
sender_26= train_data[36]["sender_labels"]
receiever_26= train_data[36]["receiver_labels"]

In [4]:
print(len(messages_26), len(sender_26), len(receiever_26))


9 9 9


In [5]:
i=0
for message in messages_26:
    print(message, sender_26[i], receiever_26[i])
    i+=1

Hello Russia. Have you worked out what you’re doing with turkey and Austria yet? I’m thinking of going anti Germany but want to see how we align in the north first. False NOANNOTATION
I sent Turkey a message but still waiting on a reply. I wanted to go anti-Germany too if you want to split the population centers? But if I don’t hear back from Turkey I may have to fortify my South flank and enter the Balkan’s even though I don’t want to. True NOANNOTATION
I moving to pressureGermany but I could use assistance on your end. Willing to share any gains that could result. True True
Yikes. You have some Turkish trouble. I’ll plan on moving into helgoland bight True NOANNOTATION
Yea, that’s always annoying playing as Russia. Trying to get AH to help out but I have to move one of my troops back West away from pressuring Germany. Oh well. True True
That’s a shame. Let’s reassess after you handle your problems in the east. True NOANNOTATION
Hey man. Let me know if you wanna support my fleet movin

In [7]:
inputs, targets = [], []

for key, instance in train_data.items():
    if isinstance(instance, dict) and 'messages' in instance:
        messages = instance['messages']
        sender_labels = instance['sender_labels']

    for idx, (message, label) in enumerate(zip(messages, sender_labels)):
        start_index = max(0, idx-10)
        end_index = min(len(messages)-1, idx)
        context = " ".join(messages[start_index:end_index])
        inputs.append(f"context: {context} classify: {message}")
        targets.append("true" if label else "false")




In [8]:
print(len(inputs), len(targets)) 
print(inputs[3])
print(targets[3])

13132 13132
context: Germany!

Just the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. 

So...do I suggest it?

I’m thinking that this is a low stakes game, not a tournament or anything, and an interesting and unusual move set might make it more fun? That’s my hope anyway.

What is your appetite like for unusual and crazy? You've whet my appetite, Italy. What's the suggestion? 👍 classify: It seems like there are a lot of ways that could go wrong...I don't see why France would see you approaching/taking Munich--while I do nothing about it--and not immediately feel skittish
true


In [9]:
test_inputs, test_targets = [], []

for key, instance in test_data.items():
    if isinstance(instance, dict) and 'messages' in instance:
        messages = instance['messages']
        sender_labels = instance['sender_labels']

    for idx, (message, label) in enumerate(zip(messages, sender_labels)):
        start_index = max(0, idx-10)
        end_index = min(len(messages)-1, idx)
        context = " ".join(messages[start_index:end_index])
        test_inputs.append(f"context: {context} classify: {message}")
        test_targets.append("true" if label else "false")

In [11]:

class DiplomacyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        assert len(self.encodings['input_ids']) == len(self.labels['input_ids']), "Encodings and labels must have the same number of samples."


    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.labels)

In [13]:
file_path = "/kaggle/input/diplomacy/validation.jsonl"
validation_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        validation_data[i] = json.loads(line)
        i+=1

In [14]:
val_texts, targetval_texts = [], []

for key, instance in validation_data.items():
    if isinstance(instance, dict) and 'messages' in instance:
        messages = instance['messages']
        sender_labels = instance['sender_labels']

    for idx, (message, label) in enumerate(zip(messages, sender_labels)):
        start_index = max(0, idx-10)
        end_index = min(len(messages)-1, idx)
        context = " ".join(messages[start_index:end_index])
        val_texts.append(f"context: {context} classify: {message}")
        targetval_texts.append("true" if label else "false")

In [16]:
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
for param in model.parameters():
    param.requires_grad = False
val_encodings = tokenizer(val_texts, return_tensors="pt", padding='max_length', max_length=512, truncation=True)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
x_embeddings = []

print(len(inputs))
for text in inputs:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Move the model to the correct device
    model.to(device)
    
    # Tokenize the input text and move the tokens to the same device as the model
    tokens = tokenizer(text, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    tokens = {key: value.to(device) for key, value in tokens.items()}
    
    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**tokens)
    
    # Access the embeddings
    x_embeddings.append(outputs.pooler_output.squeeze().cpu().numpy())  

smote = SMOTE(sampling_strategy=0.5, k_neighbors=3, random_state=42)

13132


In [18]:

X = np.stack(x_embeddings)
Y = np.array(targets)  
X_resampled, Y_resampled = smote.fit_resample(X, Y)
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_resampled)
clf = LogisticRegression(class_weight='balanced', solver='liblinear')

clf.fit(X_resampled, Y_resampled)

LogisticRegression(class_weight='balanced', solver='liblinear')

In [19]:
print("Shape of X:", X_resampled.shape)
print("Shape of Y:", Y_resampled.shape)

Shape of X: (18811, 768)
Shape of Y: (18811,)


In [20]:

# Move model to the correct device (either 'cuda' or 'cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

x_test_embeddings = []

print(len(test_inputs))

for text in test_inputs:
    # Tokenize the input text
    tokens = tokenizer(text, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    
    # Move the tokenized inputs to the same device as the model
    tokens = {key: value.to(device) for key, value in tokens.items()}
    
    with torch.no_grad():
        # Perform the forward pass
        outputs = model(**tokens)
    
    # Extract the embeddings (pooler_output) and append to the list
    x_test_embeddings.append(outputs.pooler_output.squeeze().cpu().numpy())  # Move output back to CPU if needed



2741


In [21]:
X_test = np.stack(x_test_embeddings)
Y_test = np.array(test_targets)  

y_pred = clf.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7533746807734404


In [22]:

print(classification_report(Y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(Y_test, y_pred)
print("Confusion Matrix:\n", cm)

              precision    recall  f1-score   support

       false       0.14      0.34      0.19       240
        true       0.93      0.79      0.85      2501

    accuracy                           0.75      2741
   macro avg       0.53      0.57      0.52      2741
weighted avg       0.86      0.75      0.80      2741

Confusion Matrix:
 [[  81  159]
 [ 517 1984]]


In [ ]:
joblib.dump(clf, 'logistic_regression_model.pkl')

model.save_pretrained('bert_model')
tokenizer.save_pretrained('bert_model')

print("Models saved successfully.")

240
